# Setup verification
Brief code snippets to check that the various components of the robot setup are working correctly, are connected and so on. 

In [1]:
import sys
sys.path.append("..")

from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"


## AL5D robot

### In Linux

* The power source should be plugged in and robot turned on
* This should create a serial device: /dev/ttyUSB0 (or sometimes USB1)

```sudo chown <your-account> /dev/ttyUSB0``` 

### In Windows


### In the end:

If everything is successful, this should wake up the robot, perform a short move, than turn it out

In [3]:
from robot.al5d_position_controller import PositionController

exp_robot_controller = Config().get_experiment("robot_al5d", "position_controller_00")

# Linux
# rob = PositionController("/dev/ttyUSB0") # USB1 or USB0 
# rob = PositionController(Config().values["robot"]["usb_port"]) # USB1 or USB0 
# rob = PositionController(exp_robot_controller["usb_port"])
rob = PositionController(exp_robot_controller)
# Windows (can be other COM#)
# rob = AL5D_PositionController("COM4")


***ExpRun**: Configuration for exp/run: robot_al5d/position_controller_00 successfully loaded


***ExpRun**: Configuration for exp/run: robot_al5d/pulse_controller_00 successfully loaded
***ExpRun**: The system independent config file of the experiment
	/home/al5d/WORK/BerryPicker/src/BerryPicker/src/experiment_configs/robot_al5d/angle_controller_00.yaml
	 was empty, this is a likely error, but continuing.
***ExpRun**: Configuration for exp/run: robot_al5d/angle_controller_00 successfully loaded


TypeError: 'int' object is not subscriptable

In [ ]:
pos = rob.get_position()
pos["height"] = pos["height"] + 1
rob.move(pos)

Shut down the robot: can be also used to terminate if other code dies.

In [ ]:
rob.stop_robot()

# The camera

```
pip install opencv-python
pip install opencv-contrib-python
```

Capture device 0 is usually the webcam, the other ones seem to be 2... 

On Linux, one can check the devices as 

```
/dev/video*
```

In [ ]:
import numpy as np
import cv2

In [ ]:
# identifying which cameras work
for i in range(10):
    cap = cv2.VideoCapture(i)
    ret, frame = cap.read()
    if ret:
        print(f"Camera {i} is working")
    else:
        print(f"Camera {i} not working")
    cap.release()

## Testing one single camera

Specify the camera as camera_number. Values should be one of those that show up in Linux as /dev/video*. 

If the camera with the camera_number works well, it will show up in a window. Exit the window by pressing "q" while the window being the currently active. 

In [ ]:
#camera_number = Config().values["robot"]["active_camera_list"][0]
# Experiment
camera_number = 6
print(f"Camera={camera_number}")
cap = cv2.VideoCapture(camera_number)
while(True):
    ret, frame = cap.read()
    if not ret:
        print("Capture frame returned False, exiting the loop")
        break
    # imgresized = cv2.resize(frame, (640, 480))
    imgresized = frame
    cv2.imshow(f'Camera {camera_number} - q to exit', frame)
    ret = cv2.waitKey(1)
    if ret != -1:
        ret2 = ret & 0xFF
        if ret2 == ord('q'):
            break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

## Testing multiple cameras

In all the setups tried until now (May 2024), simultaneously reading from more than 2 camera slows things down to be unusable. Apparently, some USB related problem. 

In [ ]:
DIM = (256, 256)


# create the capture devices
capture_devs = {}
# camera0 - webcam on the computer
# camera2 - right mounted
# camera3 - the free floating one
# camera4 - the center mounted one 
# cameras = [0, 2, 3, 4]
# cameras = Config().values["robot"]["active_camera_list"]
cameras = [2]
for i in cameras:
    cap = cv2.VideoCapture(i) 
    if cap is None or not cap.isOpened():
        print(f"Warning: unable to open video source: {i}")
    else:
        capture_devs[f"dev{i}"] = cap
        print(f"cap{i} works")

while True:
    images = []
    for index in capture_devs.keys():
        cap = capture_devs[index]
        ret, frame = cap.read()
        if ret:
            # FIXME: probably we want to crop as well??? 
            imgresized = cv2.resize(frame, DIM)
            images.append(imgresized)
    concatenated_image = cv2.hconcat(images)
    cv2.imshow('Press q to exit these', concatenated_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Exiting the acquisition loop as a result of a q key")
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

# The game controller
* Making sure that the game controller is working
* The version which is working with the Voyee 360 is the approxeng.input 2.5
* There was some kind of version interaction problem with version 2.6, it is unclear whether this is specific to my combinations in the hf workspace

* https://approxeng.github.io/approxeng.input/commandline.html 

```
pip install approxeng.input-2.5
```

Also note the approxeng.input-2.5 only works with Python 3.10, not higher. 

# These are the buttons that we can read out
['square']
['triangle']
['circle']
['cross']
['l1']
['r1']
['ls']
['rs']
['ls']
['rs']
['dleft']
['dright']
['dup']
['ddown']

In [ ]:
from approxeng.input.selectbinder import ControllerResource, ControllerNotFoundError
import time

try:
    with ControllerResource() as joystick:
        print('Found a joystick and connected. Press select / back to exit loop')
        print(joystick.controls)
        while joystick.connected:
            presses = joystick.check_presses()
            if len(presses.buttons) > 0:
                print(presses.names)
                print(joystick)
                print(joystick["dleft"]) # if held, returns the seconds since held
            if "select" in presses.names:
                break
except ControllerNotFoundError as e:
    print("Controller not found")
    print(e)
print("Bye")